# Imports and Defined Classes

In [ ]:
import json

class Graph:
  def __init__(self):
    # Using adjacency list structure
    self.nodes = {}

  def add_node(self, new_node):
    # Only creates a new node if it does not already exist
    if new_node not in self.nodes:
      self.nodes[new_node] = set()

  def set_adjacent(self, target_node, adjacent_node):
    # Adds adjacent node (Directed)
    self.add_node(adjacent_node)
    self.nodes[target_node].add(adjacent_node)

  def get_adjacent_nodes(self, target_node):
    # Returns adjacency list of the target node
    return self.nodes[target_node]

  def __len__(self):
    return len(self.nodes)


class TicTacToeStates:
  def __init__(self):
    self.graph = Graph()

  def Generate(self):
    # Generates states using DFS
    starting = "---------"
    print("Generating States...")
    self.__generate_states(starting, 'o')

  def __generate_states(self, current: str, move: str):
    # Base Case
    if self.check_leaf_state(current):
      return

    # Generates Next States
    self.graph.add_node(current)
    moveComplement = 'x' if move == 'o' else 'o'
    for i in range(len(current)):
      if current[i] == '-':
        new_node = current[:i] + move + current[i+1:]
        self.graph.set_adjacent(current, new_node)

        # Recursively generate states for the adjacent states
        self.__generate_states(new_node, moveComplement)

  def check_leaf_state(self, current: str):
    win_indices = [
        [0, 1, 2], [3, 4, 5], [6,7, 8],
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        [0, 4, 8], [2, 4, 6]
    ]
    # Check if any of the players has won, thus no more states are valid after this point.
    for position in win_indices:
      if all(current[i] == 'o' for i in position) or all(current[i] == 'x' for i in position):
        return True

    # Checks if there are no more moves allowed.
    if all(tile != '-' for tile in current):
      return True

    return False

  # Gets all adjacent states from the current state
  def get_adjacent_states(self, state: str):
    return self.graph.get_adjacent_nodes(state)

  def print_state(self, state: str):
    # Prints board
    print(f"|{state[0]}|{state[1]}|{state[2]}|\n|{state[3]}|{state[4]}|{state[5]}|\n|{state[6]}|{state[7]}|{state[8]}|\n")

  def print_adj_states(self, state: str):
    print("Current State:")
    self.print_state(state)

    # Prints adj states if exists
    if state not in self.graph.nodes:
      return
    print("Adjacent Nodes:")
    for node in self.graph.get_adjacent_nodes(state):
      self.print_state(node)

  def export_json(self, path = "states.json"):
    # Grabs the adjacency lists
    data = {key: list(values) for key, values in self.graph.nodes.items()}

    # Write the dictionary to a JSON file
    with open(path, "w") as json_file:
      json.dump(data, json_file, indent=4)

  def import_json(self, path = "states.json"):
    # Read the JSON data from the file
    with open(path, "r") as json_file:
      data = json.load(json_file)

    # Sets the adjacency lists
    self.graph.nodes = {key: set(values) for key, values in data.items()}

  def __len__(self):
    return len(self.graph)


class MiniMax:
  def __init__(self, states: TicTacToeStates):
    self.states = states

  # Gets the score of the current leaf state
  def get_score(self, current):
    win_indices = [
        [0, 1, 2], [3, 4, 5], [6,7, 8],
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        [0, 4, 8], [2, 4, 6]
    ]
    # Assign points to the 'o' and 'x' player.
    # 'o' is maximizing, 'x' is minimizing
    for position in win_indices:
      if all(current[i] == 'o' for i in position):
        return 512
      elif all(current[i] == 'x' for i in position):
        return -512

    # Returns 0 if a tie
    return 0

  # Mini max algorithm that determines the score of a given state
  # This is called recursively
  def mini_max(self, current_state, turn):
    if self.states.check_leaf_state(current_state):
      # At a leaf state, determine the score
      return self.get_score(current_state)
    else:
      best = None
      if turn == 'o':
        # 'o' is maximizing player
        for state in self.states.get_adjacent_states(current_state):
          score = self.mini_max(state, 'x') // 2
          if best == None or best < score:
            best = score
      else:
        # 'x' is minimizing player
        for state in self.states.get_adjacent_states(current_state):
          score = self.mini_max(state, 'o') // 2
          if best == None or best > score:
            best = score
      return best

  # Based on the adjacent board states, determine which has the best score
  # using minimax. Returns the best next board state
  def BestMove(self, current_state, turn):
    best = None
    if turn == 'o':
      # 'o' is maximizing player
      for state in self.states.get_adjacent_states(current_state):
        score = self.mini_max(state, 'x')
        # print(state, score)
        if best == None or best[0] < score:
          best = (score, state)
    else:
      # 'x' is minimizing player
      for state in self.states.get_adjacent_states(current_state):
        score = self.mini_max(state, 'o')
        # print(state, score)
        if best == None or best[0] > score:
          best = (score, state)
    return best[1] if best is not None else None


# Generate Tic Tac Toe State Space (Only need to run this once)

In [ ]:
# Generates the Tic Tac Toe State Space
States = TicTacToeStates()
States.Generate()

Generating States...


# Run the Tic Tac Toe Game

In [ ]:
# Creates a minimax object
minimax = MiniMax(States)

# Determine if the user or the computer goes first
print("'o' always goes first")
first_choice = input("User goes first? y or n: ")
print("============================================")

turn = 'o'
computer_turn = False
result = "---------"

if first_choice == 'n':
  computer_turn = True

# Game Loop
while result is None or not minimax.states.check_leaf_state(result):
  if computer_turn:
    # Determines the best move for the computer using minimax
    print("Computer is making a move...")
    result = minimax.BestMove(result, turn)

  else:
    # Selects a tile (index) from the user and error checks the input
    print("User is making a move...")
    selection_choice = int(input("Select tile: 0-8: "))
    while selection_choice >= 9 or selection_choice < 0 or result[selection_choice] != '-':
      print("Invalid tile, try again")
      selection_choice = int(input("Select tile: 0-8: "))

    # Plays the selected tile
    result = result[:selection_choice] + turn + result[selection_choice+1:]

  # Updates turn states and prints current board
  turn = 'x' if turn == 'o' else 'o'
  computer_turn = not computer_turn
  States.print_state(result)

print("Game Over!")

O always goes first
User goes first? y or n: n
Computer is making a move...
|-|-|-|
|-|-|-|
|-|o|-|

User is making a move...
Select tile: 0-8: 0
|x|-|-|
|-|-|-|
|-|o|-|

Computer is making a move...
|x|-|-|
|-|-|-|
|o|o|-|

User is making a move...
Select tile: 0-8: 10
Invalid tile, try again
Select tile: 0-8: 0
Invalid tile, try again
Select tile: 0-8: 8
|x|-|-|
|-|-|-|
|o|o|x|

Computer is making a move...
|x|-|-|
|-|o|-|
|o|o|x|

User is making a move...
Select tile: 0-8: 1
|x|x|-|
|-|o|-|
|o|o|x|

Computer is making a move...
|x|x|o|
|-|o|-|
|o|o|x|

Game Over!
